# Φόρτωση βιβλιοθηκών

Εξωτερικές βιβλιοθήκες:
 - numpy=1.21.5=py39h7a0a035_1
 - pandas=1.4.2=py39hd77b12b_0
 - liac-arff=2.5.0=pyhd3eb1b0_1

In [ ]:
import os
import glob
import pandas as pd
import numpy as np
from collections import defaultdict

# Φόρτωση δεδομένων

Από το αρχείο ***deni_(1).txt*** φορτώνουμε τυχαία ***x*** γραμμές το οποίο ορίζεται από την συνάρτηση randint με εύρος από 50 έως 150, καθαρά για τις ανάγκες των δοκιμών.Φυσικά παρακάμτουμε τις πρώτες ***7*** γραμμές καθώς δεν απαιτούνται για τους μετέπειτα υπολογισμούς.

In [ ]:
from random import randint
dataset = pd.read_csv('deni_(1).txt',skiprows=7,nrows=100,header=None)

In [ ]:
dataset

# Διαχωρισμός δεδομένων

Ο διαχωρισμός των δεδομένων στην περίπτωση των δοκιμών είναι περιττός καθώς έχουμε να επεξεργαστούμε πολύ λίγα δεδομένα. Οπότε θέτουμε την παράμετρο ***k*** της συνάρτησης ***data_seperation*** με ***0***. (Στην πραγματικότητα είναι σαν να μην εκτελείται η συνάρτηση) Σε συνθήκες όπου η συνάρτηση ***data seperation*** εκτελείται κανονικά. Σκοπός της είναι να διαλέξει εκείνα τα δεδομένα όπου το συνολικό πάτημα των πλήκτρων είναι μεγαλύτερο από ***k***. (Μπορούν να γίνουν δοκιμές όπου το k έχει πολύ μικρή τιμή π.χ ***k=2***)

In [ ]:
def data_seperation(k):
    data = dataset.values
    counter = defaultdict(int)
    for x in range(len(data)):
        button_dn =np.empty((3,0))
        if data[x][3] == 'dn':
            button_dn = data[x]
            for y in range(x+1,len(data)):
                if (button_dn[0] == data[y][0]) and (button_dn[1] == data[y][1]) and (data[y][3] =='dn'):
                    break
                elif (button_dn[0] == data[y][0]) and (button_dn[1] == data[y][1]) and (data[y][3] =='up'):
                    counter[data[x][0]]+=1
                    break
    return [key for key,value in counter.items() if value > k]

Aκολουθεί ***ταξινόμηση*** και η επιλογή των δεδομένων από ***ολόκληρο*** του dataset καθώς και η μετατροπή τους σε πίνακα της μορφή numpy

In [ ]:
seperated_data = data_seperation(0)
seperated_data.sort()
selected_data = dataset.loc[dataset[0].isin(seperated_data)]
np_selected_data = selected_data.values

# Εύρεση ζευγαριών

*Για να θεώρησουμε ότι ένα πλήκτρo έχει πατηθεί θα πρέπει η αρχική του κατάσταση είναι ***dn*** και στην συνέχεια να ακολουθεί κατάσταση ***up***.* Σε περίπτωση που μετά από κατάσταση ***dn*** ακολουθεί πάλι κατάσταση ***dn*** για το ίδιο πλήκτρο, τότε το πλήκτρο ***απορρίπτεται*** και πάμε στο ***επόμενο***. Η συνάρτηση ***find_pairs*** ουσιαστικά ομαδοποεί τις καταστάσεις των πλήκτρων, δηλάδη βρίσκει το ζευγάρι της κατάστασης ***dn*** και ***up*** του εκάστοτε πλήκτρου. Ακολουθεί παράδειγμα.


Ο παρακάτω κώδικας παρουσιάζει το τι επιστρέφει η συνάρτηση ***find_pairs*** μετά την εκτέλεση της. Ένα πίνακα ο οποίος περιέχει τα ζευγάρια των καταστάσεων. Για να θεωρηθεί ότι μια κατάσταση είναι ζευγάρι με μια άλλη θα πρέπει να έχουν κοινή ημερομηνία καθώς και κοινό αριθμό πλήκτρου. 

```python
[
    (
     array([73, '#2022-03-29#', 65878658, 'dn'], dtype=object), 
     array([73, '#2022-03-29#', 65878736,'up'],dtype=object)
    ),
    
    (
        array([83, '#2022-03-29#', 65879010, 'dn'], dtype=object),
        array([83, '#2022-03-29#', 65879088, 'up'],dtype=object)
    )
]
```

In [ ]:
def find_pairs():
    pairs=[]  
    for x in range(len(np_selected_data)):
        button_dn =np.empty((3,0))
        button_up=np.empty((3,0))
        if np_selected_data[x][3] == 'dn':
            button_dn = np_selected_data[x]
            for y in range(x+1,len(np_selected_data)):
                if (button_dn[0] == np_selected_data[y][0]) and (button_dn[1] == np_selected_data[y][1]) and (np_selected_data[y][3] =='dn'):
                    break
                elif (button_dn[0] == np_selected_data[y][0]) and (button_dn[1] == np_selected_data[y][1]) and (np_selected_data[y][3] =='up'):
                    button_up= np_selected_data[y]
                    break
        if button_dn.size !=0 and button_up.size!=0: 
            pairs.append((button_dn,button_up))
    return pairs    
                

In [ ]:
pairs = find_pairs()
pairs

# Υπολογισμός χρόνου

Το παρακάτω κελί κώδικα περιέχει την διαδικασία υπολογισμού του χρόνου των πλήκτρων από κατάσταση ***dn*** σε ***up***. Για να αποφθεχθεί το αρνητικό πρόσημο, *αφαιρούμε* από τον χρόνο ***up*** τον χρόνο ***dn***

In [ ]:
for pair in pairs:
    print(f'{pair[1][0]}[{pair[1][3]}] / {pair[0][0]} [{pair[0][3]}] : ({pair[1][3]} time) {pair[1][2]} - ({pair[0][3]} time) {pair[0][2]} = {pair[1][2] - pair[0][2]}')

Η συνάρτηση calculate_duration υπολογίζει ακριβώς ότι αναφέρθηκε πάνω και στην συνέχεια επιστρέφει τους χρόνους για κάθε πλήκτρο σε ένα dictionary

In [ ]:
def calculate_duration():
    results = defaultdict(list)
    for pair in pairs:
        up_btn_duration = int(pair[1][2])
        dn_btn_duration = int(pair[0][2])
        results[str(pair[0][0])].append(np.subtract(up_btn_duration,dn_btn_duration))
    return results

In [ ]:
duration = calculate_duration()
duration

# Υπολογισμός Μ.Ο

Ο υπολογσιμός του Μ.Ο γίνετε κάνοντας χρήση της βοηθητικής συνάρτησης ***np.average*** . Ο τελικός αριθμός θα πρέπει να αποτελείται από **2** δεκαδικά ψηφία. Η αποθήκευση των μέσων όρων γίνετε σε ένα dictionary, όπου τα κλειδιά του είναι ο αριθμός του πλήκτρου ενώ η τιμή τους ο Μ.Ο .

In [ ]:
def calculate_average():   
    average = {i:'?' for i in seperated_data}
    for key,values in duration.items():
        average.update({int(key):round(np.average(values),2)})
    return average

In [ ]:
average = calculate_average()

In [ ]:
average

# Unit - testing

Το παρακάτω unit test ελέγχει την διάρκεια καθώς και τον μέσο όρο.Οι προβλέψεις εισάγονται από τον χρήση από ένα αρχείο .csv .

In [ ]:
import unittest
from collections import OrderedDict

class TestCases(unittest.TestCase):
    
    def setUp(self):
        results = pd.read_csv("results.csv", header=None, sep=',')
        self.resulst_values = results.values
        
    def test_duration(self):
        pre_duration = res={str(value[0]):list(np.array(value[1].strip('][').split(' '),dtype=int))for value in self.resulst_values[:,[0,1]]}
        self.assertEqual(duration,pre_duration)
        
    def test_average(self):
        pre_average = dict(OrderedDict(sorted(dict(self.resulst_values[:,[0,2]]).items())))
        self.assertEqual(average,pre_average)


unittest.main(argv=[''], exit=False)